In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
#Import basic libraries
%matplotlib inline


import os
import sys
import settings
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from django_pandas.io import read_frame
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates

from matplotlib.dates import date2num

In [3]:
#Prepare to load stock data as pandas dataframe from source. In this case, prepare django
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from stocks.models import Listing, Stock, Market

In [ ]:
#Asian paints
stock = Stock.objects.get(sid='ASIANPAINT') #Load Nifty

print (stock.sid)
print(stock.name)
print (stock)


In [ ]:
#Read data into a dataframe
listings = Listing.objects.filter(stock=stock)
df = read_frame(listings, index_col='date')
for column in df.columns:
    if column != 'stock':
       df[column] = pd.to_numeric(df[column]) 
df = df.sort_index()
df = df.reindex(columns = ['opening', 'high', 'low', 'closing', 'traded'])
df.rename(columns={"opening": "Open", "high": "High", "low": "Low", "closing":"Close", "traded":"Volume"}, inplace=True)

In [ ]:
#Optionally, filter out by date range
start_date = '2020-01-01'
end_date = '2021-12-31'
ticker = 'ASIANPAINTS'

df = df.loc[start_date:end_date]

In [ ]:
df.head()

In [4]:
#Import TA-lib and backtesting library
import talib
from talib.abstract import *
from talib import MA_Type

from backtesting import Strategy
from backtesting.lib import crossover

/mnt/c/Users/anshu/Documents/experimental/env/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [ ]:
#Create a strategy
class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 20
    n2 = 200
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, int(self.n1))
        self.sma2 = self.I(SMA, self.data.Close, int(self.n2))
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [ ]:
from backtesting import Backtest

bt = Backtest(df, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

In [ ]:
bt.plot()

In [ ]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(20, 200, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

In [ ]:
stats._strategy

In [ ]:
bt.plot(plot_volume=False, plot_pl=False)

In [ ]:
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy


class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, int(self.n1))
        sma2 = self.I(SMA, self.data.Close, int(self.n2))
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [ ]:
bt = Backtest(df, SmaCross, commission=.002)
bt.run()
bt.plot()

In [ ]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(20, 200, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

In [ ]:
stats._strategy

In [ ]:
bt.plot(plot_volume=False, plot_pl=False)

In [ ]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 14  # Daily RSI lookback periods
    w_rsi = 14  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, int(10))
        self.ma20 = self.I(SMA, self.data.Close, int(20))
        self.ma50 = self.I(SMA, self.data.Close, int(50))
        self.ma100 = self.I(SMA, self.data.Close, int(100))
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(talib.RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', talib.RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [ ]:
backtest = Backtest(df, System, commission=.002)
backtest.run()

In [ ]:
backtest.plot()

In [ ]:
%%time

backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))

In [ ]:
backtest.plot()

 `argrelextrema` is used for detecting peaks in SciPy's signal processing library, and `deque` is like a fixed-length list that will drop the oldest entry and keep the new ones if you exceed its length. We'll use the first to spot our extrema in our data, then cycle through them and keep the points that are higher than the previous entries.

To spot our extrema, we need to pass an argument called order. This defines how many points on either side of our peak we need to actually label something a peak. So with order=5, we need something to be the highest point within 5-data points to the right and left. The other argument we provide is K, which is simply an integer to determine how many consecutive peaks we want to identify to determine a trend of higher highs.

In [ ]:
#RSI Divergence strategy
#https://raposa.trade/blog/test-and-trade-rsi-divergence-in-python

from scipy.signal import argrelextrema
from collections import deque

def getHigherLows(data: np.array, order=5, K=2):
  '''
  Finds consecutive higher lows in price pattern.
  Must not be exceeded within the number of periods indicated by the width 
  parameter for the value to be confirmed.
  K determines how many consecutive lows need to be higher.
  '''
  # Get lows
  low_idx = argrelextrema(data, np.less, order=order)[0]
  lows = data[low_idx]
  # Ensure consecutive lows are higher than previous lows
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(low_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if lows[i] < lows[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def getLowerHighs(data: np.array, order=5, K=2):
  '''
  Finds consecutive lower highs in price pattern.
  Must not be exceeded within the number of periods indicated by the width 
  parameter for the value to be confirmed.
  K determines how many consecutive highs need to be lower.
  '''
  # Get highs
  high_idx = argrelextrema(data, np.greater, order=order)[0]
  highs = data[high_idx]
  # Ensure consecutive highs are lower than previous highs
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(high_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if highs[i] > highs[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def getHigherHighs(data: np.array, order=5, K=2):
  '''
  Finds consecutive higher highs in price pattern.
  Must not be exceeded within the number of periods indicated by the width 
  parameter for the value to be confirmed.
  K determines how many consecutive highs need to be higher.
  '''
  # Get highs
  high_idx = argrelextrema(data, np.greater, order=5)[0]
  highs = data[high_idx]
  # Ensure consecutive highs are higher than previous highs
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(high_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if highs[i] < highs[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema

def getLowerLows(data: np.array, order=5, K=2):
  '''
  Finds consecutive lower lows in price pattern.
  Must not be exceeded within the number of periods indicated by the width 
  parameter for the value to be confirmed.
  K determines how many consecutive lows need to be lower.
  '''
  # Get lows
  low_idx = argrelextrema(data, np.less, order=order)[0]
  lows = data[low_idx]
  # Ensure consecutive lows are lower than previous lows
  extrema = []
  ex_deque = deque(maxlen=K)
  for i, idx in enumerate(low_idx):
    if i == 0:
      ex_deque.append(idx)
      continue
    if lows[i] > lows[i-1]:
      ex_deque.clear()

    ex_deque.append(idx)
    if len(ex_deque) == K:
      extrema.append(ex_deque.copy())

  return extrema


In [ ]:
from matplotlib.lines import Line2D # For legend
price = df['Close'].values
dates = df.index
# Get higher highs, lower lows, etc.
order = 5
hh = getHigherHighs(price, order)
lh = getLowerHighs(price, order)
ll = getLowerLows(price, order)
hl = getHigherLows(price, order)
# Get confirmation indices
hh_idx = np.array([i[1] + order for i in hh])
lh_idx = np.array([i[1] + order for i in lh])
ll_idx = np.array([i[1] + order for i in ll])
hl_idx = np.array([i[1] + order for i in hl])
# Plot results
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.figure(figsize=(12, 8))
plt.plot(df['Close'])
plt.scatter(dates[hh_idx], price[hh_idx-order], marker='^', c=colors[1])
plt.scatter(dates[lh_idx], price[lh_idx-order], marker='v', c=colors[2])
plt.scatter(dates[ll_idx], price[ll_idx-order], marker='v', c=colors[3])
plt.scatter(dates[hl_idx], price[hl_idx-order], marker='^', c=colors[4])
_ = [plt.plot(dates[i], price[i], c=colors[1]) for i in hh]
_ = [plt.plot(dates[i], price[i], c=colors[2]) for i in lh]
_ = [plt.plot(dates[i], price[i], c=colors[3]) for i in ll]
_ = [plt.plot(dates[i], price[i], c=colors[4]) for i in hl]
plt.xlabel('Date')
plt.ylabel('Price (Rs)')
plt.title(f'Potential Divergence Points for {ticker} Closing Price')
legend_elements = [
  Line2D([0], [0], color=colors[0], label='Close'),
  Line2D([0], [0], color=colors[1], label='Higher Highs'),
  Line2D([0], [0], color='w',  marker='^',
         markersize=10,
         markerfacecolor=colors[1],
         label='Higher High Confirmation'),
  Line2D([0], [0], color=colors[2], label='Higher Lows'),
  Line2D([0], [0], color='w',  marker='^',
         markersize=10,
         markerfacecolor=colors[2],
         label='Higher Lows Confirmation'),
  Line2D([0], [0], color=colors[3], label='Lower Lows'),
  Line2D([0], [0], color='w',  marker='v',
         markersize=10,
         markerfacecolor=colors[3],
         label='Lower Lows Confirmation'),
  Line2D([0], [0], color=colors[4], label='Lower Highs'),
  Line2D([0], [0], color='w',  marker='^',
         markersize=10,
         markerfacecolor=colors[4],
         label='Lower Highs Confirmation')
]
plt.legend(handles=legend_elements, bbox_to_anchor=(1, 0.65))
plt.show()

In [ ]:
#Compute the RSI of the data
rsi_duration = 14;
rsi = talib.RSI(df['Close'].values, rsi_duration)
df['RSI'] =  rsi

rsi_hh = getHigherHighs(rsi, order)
rsi_lh = getLowerHighs(rsi, order)
rsi_ll = getLowerLows(rsi, order)
rsi_hl = getHigherLows(rsi, order)

# Get confirmation indices
rsi_hh_idx = np.array([i[1] + order for i in rsi_hh])
rsi_lh_idx = np.array([i[1] + order for i in rsi_lh])
rsi_ll_idx = np.array([i[1] + order for i in rsi_ll])
rsi_hl_idx = np.array([i[1] + order for i in rsi_hl])

In [ ]:
fig, ax = plt.subplots(2, figsize=(20, 12), sharex=True)
ax[0].plot(df['Close'])
ax[0].scatter(dates[hh_idx], price[hh_idx-order], 
              marker='^', c=colors[1])
ax[0].scatter(dates[lh_idx], price[lh_idx-order],
              marker='v', c=colors[2])
ax[0].scatter(dates[hl_idx], price[hl_idx-order],
              marker='^', c=colors[3])
ax[0].scatter(dates[ll_idx], price[ll_idx-order],
              marker='v', c=colors[4])
_ = [ax[0].plot(dates[i], price[i], c=colors[1]) for i in hh]
_ = [ax[0].plot(dates[i], price[i], c=colors[2]) for i in lh]
_ = [ax[0].plot(dates[i], price[i], c=colors[3]) for i in hl]
_ = [ax[0].plot(dates[i], price[i], c=colors[4]) for i in ll]
ax[0].set_ylabel('Price (Rs)')
ax[0].set_title(f'Price and Potential Divergence Points for {ticker}')
ax[0].legend(handles=legend_elements)
ax[1].plot(df['RSI'])
ax[1].scatter(dates[rsi_hh_idx], rsi[rsi_hh_idx-order], 
              marker='^', c=colors[1])
ax[1].scatter(dates[rsi_lh_idx], rsi[rsi_lh_idx-order],
              marker='v', c=colors[2])
ax[1].scatter(dates[rsi_hl_idx], rsi[rsi_hl_idx-order],
              marker='^', c=colors[3])
ax[1].scatter(dates[rsi_ll_idx], rsi[rsi_ll_idx-order],
              marker='v', c=colors[4])
_ = [ax[1].plot(dates[i], rsi[i], c=colors[1]) for i in rsi_hh]
_ = [ax[1].plot(dates[i], rsi[i], c=colors[2]) for i in rsi_lh]
_ = [ax[1].plot(dates[i], rsi[i], c=colors[3]) for i in rsi_hl]
_ = [ax[1].plot(dates[i], rsi[i], c=colors[4]) for i in rsi_ll]
ax[1].set_ylabel('RSI')
ax[1].set_title(f'RSI and Potential Divergence Points for {ticker}')
ax[1].set_xlabel('Date')
plt.tight_layout()
plt.show() 

In [ ]:
#RSI Divergence strategy
def getHHIndex(data: np.array, order=5, K=2):
  extrema = getHigherHighs(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getLHIndex(data: np.array, order=5, K=2):
  extrema = getLowerHighs(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getLLIndex(data: np.array, order=5, K=2):
  extrema = getLowerLows(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getHLIndex(data: np.array, order=5, K=2):
  extrema = getHigherLows(data, order, K)
  idx = np.array([i[-1] + order for i in extrema])
  return idx[np.where(idx<len(data))]

def getPeaks(data, key='Close', order=5, K=2):
  vals = data[key].values
  hh_idx = getHHIndex(vals, order, K)
  lh_idx = getLHIndex(vals, order, K)
  ll_idx = getLLIndex(vals, order, K)
  hl_idx = getHLIndex(vals, order, K)

  data[f'{key}_highs'] = np.nan
  data[f'{key}_highs'][hh_idx] = 1
  data[f'{key}_highs'][lh_idx] = -1
  data[f'{key}_highs'] = data[f'{key}_highs'].ffill().fillna(0)
  data[f'{key}_lows'] = np.nan
  data[f'{key}_lows'][ll_idx] = 1
  data[f'{key}_lows'][hl_idx] = -1
  data[f'{key}_lows'] = data[f'{key}_highs'].ffill().fillna(0)
  return data


In [ ]:
from backtesting import Strategy
def getPeaks(data, key='Close', order=5, K=2):
    
    vals = data[key].values
    hh_idx = getHHIndex(vals, order, K)
    lh_idx = getLHIndex(vals, order, K)
    
    a = np.empty(vals.shape)
    a[:] = np.nan
    a[hh_idx] = 1
    a[lh_idx] = -1
    #a = a.ffill().fillna(0)
    a[np.isnan(a)] = 0
    return a

def getValleys(data, key='Close', order=5, K=2):
    vals = data[key].values
    ll_idx = getLLIndex(vals, order, K)
    hl_idx = getHLIndex(vals, order, K)
    
    a = np.empty(vals.shape)
    a[:] = np.nan
    a[ll_idx] = 1
    a[hl_idx] = -1
    #a = data[f'{key}_highs'].ffill().fillna(0)
    a[np.isnan(a)] = 0
    return a
    
#Create a strategy
class RsiDivergence(Strategy):
    '''
      Go long/short on price and RSI divergence.
      - Long if price to lower low and RSI to higher low with RSI < 50
      - Short if price to higher high and RSI to lower high with RSI > 50
      Sell if divergence disappears.
      Sell if the RSI crosses the centerline.
    '''
    rsi_level = 50
    order = 5
    K = 2
    rsi_period = 14
    
    ema_s = 50
    ema_l = 200
    
    
    def init(self):
        # Precompute the RSI
        self.ema_s = self.I(EMA, self.data.df['Close'], int(self.ema_s))
        self.ema_l = self.I(EMA, self.data.df['Close'], int(self.ema_l))
        self.data.df['RSI'] = talib.RSI(self.data.df['Close'], self.rsi_period)
        self.rsi = self.I(talib.RSI, self.data.df['Close'], self.rsi_period)
        
        self.entry_rsi = 0
        
        self.close_highs = self.I(getPeaks, self.data.df, key='Close', order=self.order, K=self.K)
        self.close_lows = self.I(getValleys, self.data.df, key='Close', order=self.order, K=self.K)
        
        self.rsi_highs = self.I(getPeaks, self.data.df, key='RSI', order=self.order, K=self.K)
        self.rsi_lows = self.I(getValleys, self.data.df, key='RSI', order=self.order, K=self.K)
    
    def next(self):
        price = self.data.Close[-1]
        if np.isnan(self.data.RSI[-1]):
            return
        
        if self.position is None: #We might be opening a position
            #Long if RSI is less than 50 and making higher high
            if self.close_lows[-1] == -1 and self.rsi_lows == 1: #Price makes lower low, indicator makes higher low (Positive divergence)
                if self.data.RSI[-1] < 50:
                    self.entry_rsi = self.data.RSI[-1].copy()
                    #self.buy(sl=.92 * price)
                    self.buy()       
            # Short if price to higher high and indicator to lower high
            elif self.close_highs[-1] == 1 and self.rsi_highs[-1] == -1: #Price making higher high, RSI making lower high (Negative divergence)
                if row['RSI'] > 50:
                    self.sell()
                    self.entry_rsi = self.data.RSI[-1].copy()
        # If current position is long
        elif self.position.is_long:
            if self.rsi[-1] < 50 and self.rsi[-1] < self.entry_rsi:
                pass
            else:
                self.position.close()
            #elif self.ema_s > self.ema_l:
            #    pass

        # If current position is short
        elif self.position.is_short:
            if self.rsi[-1] > 50 and self.rsi[-1] < self.entry_rsi:
                pass
            else:
                self.position.close()
                

In [ ]:
from backtesting import Backtest

bt = Backtest(df, RsiDivergence, cash=10_000, commission=.002)
stats = bt.run()
stats


In [ ]:
bt.plot()

## Bollinger band and RSI strategy

In [ ]:
#https://kylelix7.github.io/Trading-Strategy-Technical-Analysis-with-Python-TA-Lib/
#Create a strategy
class BollingerBand(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    timeperiod = 20
    rsi_period = 14
    nbdevup = 2
    nbdevdn = 2
    
    def init(self):
        # Precompute the two moving averages
        self.rsi = self.I(talib.RSI, self.data.Close, int(self.rsi_period))
        self.up, self.mid, self.low = self.I(talib.BBANDS, self.data.Close, 
                                              timeperiod = int(self.timeperiod), 
                                              nbdevup = int(self.nbdevup),
                                             nbdevdn = int(self.nbdevdn),
                                             matype=0)
    
    def next(self):
        bbp = (self.data.Close[-1] - self.low[-1]) / (self.up[-1] - self.low[-1])
        if self.rsi[-1]>70 and bbp>1:
            self.position.close()
            self.sell()

        if self.rsi[-1]<30 and bbp<0:
            self.position.close()
            self.buy()

In [ ]:
from backtesting import Backtest

bt = Backtest(df, BollingerBand, cash=10_000, commission=.002)
stats = bt.run()
stats

## EMA Trend Following Scalping strategy on intraday

In [3]:
#Import TA-lib and backtesting library
import talib
from talib.abstract import *
from talib import MA_Type

from backtesting import Strategy
from backtesting.lib import crossover

/mnt/c/Users/anshu/Documents/experimental/env/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [13]:
#Load Nifty data
df = pd.read_csv('./NIFTY50.csv', index_col=0)
#df = pd.read_csv('./NIFTY50.csv')
for column in df.columns:
    if column != 'date':
       df[column] = pd.to_numeric(df[column])
#df['date'] = pd.to_datetime(df['date'])
#df['date'] = df['date'].tz_localize(None)
#df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df.index = df.index.tz_localize(None)

df = df.sort_index()
df = df.reindex(columns = ['open', 'high', 'low', 'close', 'volume'])
#df.drop('volume', axis=1, inplace=True)
df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close":"Close", "volume":'Volume'}, inplace=True)

df = df[~df.index.duplicated(keep='first')]

print(df.tail())
#Optionally, filter out by date range
start_date = '2022-02-18 09:15:00'
end_date = '2022-02-18 15:30:00'
df = df.loc[start_date:end_date]
#drop duplicates


                         Open      High       Low     Close  Volume
date                                                               
2022-02-18 15:25:00  17267.90  17276.35  17265.90  17276.35       0
2022-02-18 15:26:00  17276.85  17277.25  17273.40  17274.65       0
2022-02-18 15:27:00  17273.85  17276.25  17272.10  17274.45       0
2022-02-18 15:28:00  17274.60  17274.65  17263.10  17266.00       0
2022-02-18 15:29:00  17265.95  17271.30  17263.65  17268.75       0


In [14]:
#print(df.index.dtype)
print(df.head())

                         Open      High       Low     Close  Volume
date                                                               
2022-02-18 09:15:00  17236.05  17274.00  17220.20  17264.15       0
2022-02-18 09:16:00  17263.15  17274.20  17246.25  17253.65       0
2022-02-18 09:17:00  17253.45  17278.45  17253.45  17276.60       0
2022-02-18 09:18:00  17277.70  17285.55  17275.65  17283.05       0
2022-02-18 09:19:00  17283.10  17284.45  17263.50  17263.50       0


In [ ]:
class EMACrossoverStrategy(Strategy):
    '''
    Naive:
    If the price is below the EMA, trend is down, else up.
    
    If price crosses upwards, go long
    If price crosses downwards, go short
    '''
    EMA_period = 5
    
    def init(self):
        # Precompute the moving average
        self.ema = self.I(talib.EMA, self.data.Close, int(self.EMA_period))
        
        self.stoploss = self.data.Close[0]
        #print('First EMA: {}'.format(self.ema[0]))
    
    def next(self):
        #print('Active trades:')
        #print(self.trades)
        
        #print('Closed trades:')
        #print(self.closed_trades)
        closed = False
        
        #Has the price crossed over?
        if crossover(self.data.Close, self.ema):
            #print(f"Cross: {self.ema[-1]}")
            #print('Up', self.data.index[-1], self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1] )
            if self.position and self.position.is_short:
                self.position.close()
                #print(f'Close {self.data.Close[-1]} ({self.position.pl})')
                closed = True
            if not self.position or closed:
                self.stoploss = self.data.Low[-1]
                #self.buy(sl=self.stoploss)
                self.buy()
                #print(f'Buy {self.data.Close[-1]} \tSL {self.stoploss}')
            else:
                #print(f'Have position Long:{self.position.is_long}')
                pass
        elif crossover(self.ema, self.data.Close):
            #print('Down', self.data.index[-1], self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1] )
            if self.position and self.position.is_long:
                self.position.close()
                #print(f'Close {self.data.Close[-1]} ({self.position.pl})')
                closed = True
            if not self.position or closed:
                self.stoploss = self.data.High[-1]
                #self.sell(sl=self.stoploss)
                self.sell()
                #print(f'Sell {self.data.Close[-1]} \tSL {self.stoploss}')
            else:
                #print(f'Have position Short:{self.position.is_short}')
                pass

In [ ]:
from backtesting import Backtest

bt = Backtest(df, EMACrossoverStrategy, cash=1000000, commission=40/1000000)
stats = bt.run()
print(stats)

#bt.plot()

In [ ]:
%%time

stats = bt.optimize(EMA_period=range(5, 200, 5),
                    maximize='Equity Final [$]',
                    constraint=None)
stats

In [ ]:
stats._strategy

So, simple EMA following would give outrageous returns if there is 0 commission. But, adding a bit of commission, values drop ridiculously.

Following EMA on 5 time period level (5 ticks on 1-minute data):

Commission = 0 : Returns: 69047%

Commission = 20/1000000 : Returns: 2366%

Commission = 30/1000000 : Returns: 366%

Commission = 40/1000000 : Returns: -11.6%

One could attempt an answer: During periods of low volatility, the EMA crossover rate would be quite high. So, one wants to keep a margin while setting stop-losses. 

One of the simplest ways could be using the high/low of the previous candle when going short/long.

In [ ]:
class EMACrossover_SL_Strategy(Strategy):
    '''
    Naive:
    If the price is below the EMA, trend is down, else up.
    
    If price crosses upwards, go long with candle low as stoploss
    If price crosses downwards, go short with candle high as stoploss
    '''
    EMA_period = 5
    
    def init(self):
        # Precompute the moving average
        self.ema = self.I(talib.EMA, self.data.Close, int(self.EMA_period))
        
        self.stoploss = self.data.Close[0]
        #print('First EMA: {}'.format(self.ema[0]))
    
    def next(self):
        #print('Active trades:')
        #print(self.trades)
        
        #print('Closed trades:')
        #print(self.closed_trades)
        closed = False
        
        #Has the price crossed over?
        if crossover(self.data.Close, self.ema):
            #print(f"Cross: {self.ema[-1]}")
            print('Up', self.data.index[-1], self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1] )
            if self.position and self.position.is_short and self.data.Close[-1] > self.stoploss:
                self.position.close()
                print(f'Close {self.data.Close[-1]} ({self.position.pl})')
                closed = True
            if not self.position or closed:
                self.stoploss = self.data.Low[-1]
                #self.buy(sl=self.stoploss)
                self.buy()
                print(f'Buy {self.data.Close[-1]} \tSL {self.stoploss}')
            else:
                #print(f'Have position Long:{self.position.is_long}')
                pass
        elif crossover(self.ema, self.data.Close):
            print('Down', self.data.index[-1], self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1] )
            if self.position and self.position.is_long and self.data.Close[-1] < self.stoploss:
                self.position.close()
                print(f'Close {self.data.Close[-1]} ({self.position.pl})')
                closed = True
            if not self.position or closed:
                self.stoploss = self.data.High[-1]
                #self.sell(sl=self.stoploss)
                self.sell()
                print(f'Sell {self.data.Close[-1]} \tSL {self.stoploss}')
            else:
                #print(f'Have position Short:{self.position.is_short}')
                pass
        else:
            if self.position:
                if self.position.is_long and self.data.Close[-1] < self.stoploss:
                    print(self.data.index[-1], self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1] )
                    self.position.close()
                    print(f'Close long {self.data.Close[-1]} ({self.position.pl})')
                    #self.stoploss = self.data.High[-1]
                    #self.sell()
                if self.position.is_short and self.data.Close[-1] > self.stoploss:
                    print(self.data.index[-1], self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1] )
                    self.position.close()
                    print(f'Close short {self.data.Close[-1]} ({self.position.pl})')
                    #self.stoploss = self.data.Low[-1]
                    #self.buy()
from backtesting import Backtest

bt = Backtest(df, EMACrossover_SL_Strategy, cash=1000000, commission=0/1000000)
stats = bt.run()
print(stats)

bt.plot()

Following EMA on 5 time period level (5 ticks on 1-minute data):

Commission = 0 : Returns: 4%

Commission = 20/1000000 : Returns: 3.8%

Commission = 30/1000000 : Returns: 3.7%

Commission = 40/1000000 : Returns: 0.35%


Here, we see that the performance with commission improves a little in the worst case, but drops steeply in other cases. The major reason is that the SL is a fixed value and we will, sooner or later, get to that value. So, we want a moving stop-loss target once the price has moved in our favor.


Or, one could hypothesize that if we have the range within a window period, one might set the edges of the range as stop-losses. One could consider using the Average-True-Range (ATR) indicator for that.

In [ ]:
class EMACrossover_ATR_Strategy(Strategy):
    '''
    Naive:
    If the price is below the EMA, trend is down, else up.
    
    If price crosses upwards, go long with candle low as stoploss
    If price crosses downwards, go short with candle high as stoploss
    '''
    EMA_period = 5
    
    def init(self):
        # Precompute the moving average
        self.ema = self.I(talib.EMA, self.data.Close, int(self.EMA_period))
        
        self.stoploss = self.data.Close[0]
        #print('First EMA: {}'.format(self.ema[0]))
    
    def next(self):
        #print('Active trades:')
        #print(self.trades)
        
        #print('Closed trades:')
        #print(self.closed_trades)
        closed = False
        
        #Has the price crossed over?
        if crossover(self.data.Close, self.ema):
            #print(f"Cross: {self.ema[-1]}")
            #print('Up', self.data.index[-1], self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1] )
            if self.position and self.position.is_short:
                self.position.close()
                #print(f'Close {self.data.Close[-1]} ({self.position.pl})')
                closed = True
            if not self.position or closed:
                self.stoploss = self.data.Low[-1]
                #self.buy(sl=self.stoploss)
                self.buy()
                #print(f'Buy {self.data.Close[-1]} \tSL {self.stoploss}')
            else:
                #print(f'Have position Long:{self.position.is_long}')
                pass
        elif crossover(self.ema, self.data.Close):
            #print('Down', self.data.index[-1], self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1] )
            if self.position and self.position.is_long:
                self.position.close()
                #print(f'Close {self.data.Close[-1]} ({self.position.pl})')
                closed = True
            if not self.position or closed:
                self.stoploss = self.data.High[-1]
                #self.sell(sl=self.stoploss)
                self.sell()
                #print(f'Sell {self.data.Close[-1]} \tSL {self.stoploss}')
            else:
                #print(f'Have position Short:{self.position.is_short}')
                pass

In [ ]:
class EMAStrategy(Strategy):
    '''
    Naive:
    If the price is below the EMA, trend is down, else up.
    
    If price touches EMA line:
     - take note of trend, 
     - and price (might be retest, might be trend change)
    If price reverses:
     - set trend as confirmed
     - If no position exists:
         - take position
     - If position exists:
         - update stop loss as [(EMA+-tolerance), Last candle's high/low]
    If price does not reverse:
     - set trend to opposite
     - If position exists:
         - If price hits stop-loss, exit position
    
    '''
    EMA_period = 20
    
    def init(self):
        # Precompute the moving average
        self.ema = self.I(talib.EMA, self.data.Close, int(self.EMA_period))
        
        self.direction = 0 #1 is up, -1 is down
        self.confirm = 0
        self.stoploss = self.data.Close[0]
        
        self.distance_from_last_sl = 0
        
        if self.data.Close[0] > self.ema[0]:
            self.direction = 1
        else:
            self.direction = -1
    
    def next(self):
        #Has the price crossed over?
        if crossover(self.data.Close, self.ema):
            if self.direction == 1:
                self.confirm  = 1 #We were in uptrend and price reversed to upside, uptrend confirmed (again)
                if not self.position:
                    self.buy()
                if self.position.is_long:
                    self.stoploss = min(self.data.Low[self.distance_from_last_sl:-1]
                    self.distance_from_last_sl = 0
                    
            elif self.direction == -1:
                self.confirm = 0 #We were in downtrend and price is piercing upwards. Could be a reversal or retest
                
            '''
            if self.direction == -1 and self.confirm == 0: #Could be confirmation
                pass
            elif self.direction <= 0 and self.confirm == 0: #Might break to upside
                self.direction = 1
                self.confirm = 0
                #self.position.close() #Close any bearish positions
            elif self.direction > 0 and self.confirm == 0: #Confirmed uptrend
                self.confirm = 1
                self.buy()
            elif self.direction > 0 and self.confirm == 1: #Retest
                self.buy() #Add more or hold
            elif self.direction < 0 and self.confirm == 1: #Close bearish position now
                self.position.close()
            '''
        elif crossover(self.ema, self.data.Close):
            if self.direction == 1:
                self.confirm = 0 #We were in uptrend and price is piercing downwards. Could be a reversal or retest
            elif self.direction == -1:
                self.confirm = 1 #Downtrend confirmed (again)
                if not self.position:
                    self.sell()
                if self.position.is_short:
                    self.stoploss = max(self.data.High[self.distance_from_last_sl:-1])
                    self.distance_from_last_sl = 0
            '''
            if self.direction == 1 and self.confirm == 0: #Could be confirmation
                pass
            elif self.direction <= 0 and self.confirm == 0: #Might break to downside
                self.direction = -1
                self.confirm = 0
                #self.position.close() #Close any bullish positions
            elif self.direction < 0 and self.confirm == 0: #Confirmed downtrend
                self.confirm = 1
                self.sell()
            elif self.direction < 0 and self.confirm == 1: #Retest
                self.sell() #Short more or hold
            elif self.direction > 0 and self.confirm == 1: #Close bullish position now
                self.position.close()
            '''
        else:
            self.distance_from_last_sl -= 1 #Increase distance
        if self.direction == 1 and self.data.Close[-1] < self.stoploss:
            self.position.close()
        if self.direction == -1 and self.data.Close[-1] > self.stoploss:
            self.position.close()

In [ ]:
from backtesting import Backtest

bt = Backtest(df, EMAStrategy, cash=10_000, commission=.002)
stats = bt.run()
stats

I have noticed that a stock hitting the 20EMA or 200EMA tends to reverse in its first attempt. We want to quantify the claim.

In [6]:
df = pd.read_csv('./ICICIBANK.csv', index_col=0)
#df = pd.read_csv('./NIFTY50.csv')
for column in df.columns:
    if column != 'date':
       df[column] = pd.to_numeric(df[column])
#df['date'] = pd.to_datetime(df['date'])
#df['date'] = df['date'].tz_localize(None)
#df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df.index = df.index.tz_localize(None)

df = df.sort_index()
df = df.reindex(columns = ['open', 'high', 'low', 'close', 'volume'])
#df.drop('volume', axis=1, inplace=True)
df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close":"Close", "volume":'Volume'}, inplace=True)

df = df[~df.index.duplicated(keep='first')]
#Optionally, filter out by date range
#print(df.head(10))
filter = False
if filter:
    start_date = '2015-02-01 09:15:00'
    end_date = '2015-03-01 03:30:00'
    df = df.loc[start_date:end_date]

df.head()

,Open,High,Low,Close,Volume
date,,,,,
2015-01-01,320.64,321.32,318.36,320.27,2583281
2015-01-02,320.00,330.45,320.00,329.36,8272990
2015-01-05,332.09,334.00,328.64,330.05,10316090
2015-01-06,326.73,327.18,314.73,316.05,13282427
2015-01-07,314.18,314.77,304.95,307.50,21254332


In [31]:
class Rebound_Strategy(Strategy):
    '''
    ema_period = EMA period
    bound = proximity
    target = target retracement
    If price above ema_period and reaching ema_period, short near bound with an equal stop-loss and target as immediate target retracement
    
    '''
    ema_period = 20
    bound = 3 #0.5%
    target_val = 30 #3% denominator = 1000
    base = 1000.0
    
    def init(self):
        # Precompute the moving average
        self.ema = self.I(talib.EMA, self.data.Close, int(self.ema_period))
        self.direction = 0
        self.target = 0
        self.stoploss = 0
        
        self.sell_long = True
        self.sell_short = True

    def next(self):
        if not pd.isna(self.ema[-1]):
            if self.sell_long:
                if self.data.Low[-1] > self.ema[-1] and \
                    ((self.data.Close[-1] - self.ema[-1])/self.data.Close[-1])>(int(self.bound)/self.base):
                    self.direction = 1
                #elif self.direction != 0 and self.data.Low[-1] > self.ema[-1] and \
                elif self.direction == 1 and (self.data.Close[-1] >= self.ema[-1]) and\
                    ((self.data.Close[-1] - self.ema[-1])/self.data.Close[-1])<=(int(self.bound)/self.base):
                    #print('Down', self.data.index[-1], self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1] )
                    self.direction = 0
                    self.target = (1.0 + (int(self.target_val)/self.base)) * self.data.Close[-1]
                    self.stoploss = self.data.Close[-1] - 2*((int(self.bound)/self.base) *self.data.Close[-1])  
                    #print(f'Target: {self.target}, SL: {self.stoploss}')
                    self.buy(sl = self.stoploss, tp = self.target)
                elif self.direction == 1 and self.data.Close[-1] < self.ema[-1] and \
                    ((self.ema[-1] - self.data.Close[-1])/self.data.Close[-1])<=(int(self.bound)/self.base):
                    #print('Down', self.data.index[-1], self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1] )
                    self.direction = 0
                    self.target = (1.0 + (int(self.target_val)/self.base)) * self.data.Close[-1]
                    self.stoploss = self.data.Close[-1] - 1*((int(self.bound)/self.base) *self.ema[-1])  
                    #print(f'Target: {self.target}, SL: {self.stoploss}')
                    self.buy(sl = self.stoploss, tp = self.target)
            if self.sell_short:
                if self.data.High[-1] < self.ema[-1] and \
                    ((self.ema[-1] - self.data.Close[-1])/self.data.Close[-1])>(int(self.bound)/self.base):
                    self.direction = -1
                #elif self.direction != 0 and self.data.High[-1] < self.ema[-1] and \
                elif self.direction == -1  and (self.data.Close[-1] <= self.ema[-1]) and \
                    ((self.ema[-1] - self.data.Close[-1])/self.data.Close[-1])<=(int(self.bound)/self.base):
                    #print('Up', self.data.index[-1], self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1] )
                    self.direction = 0
                    self.target = (1-(int(self.target_val)/self.base)) * self.data.Close[-1]
                    self.stoploss = self.data.Close[-1] + 2*((self.bound/self.base)*self.data.Close[-1])
                    #print(f'Target: {self.target}, SL: {self.stoploss}')
                    self.sell(sl = self.stoploss, tp = self.target)
                elif self.direction == -1  and (self.data.Close[-1] > self.ema[-1]) and\
                    ((self.data.Close[-1] - self.ema[-1])/self.data.Close[-1])<=(int(self.bound)/self.base):
                    #print('Up', self.data.index[-1], self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1] )
                    self.direction = 0
                    self.target = (1-(int(self.target_val)/self.base)) * self.data.Close[-1]
                    self.stoploss = self.data.Close[-1] + 1*((self.bound/self.base)*self.ema[-1])
                    #print(f'Target: {self.target}, SL: {self.stoploss}')
                    self.sell(sl = self.stoploss, tp = self.target)
                
from backtesting import Backtest
bt = Backtest(df, Rebound_Strategy, cash=1000000, commission=0/1000000, trade_on_close=False)
stats = bt.run()
print(stats)

bt.plot()

Start                     2015-01-01 00:00:00
End                       2022-02-18 00:00:00
Duration                   2605 days 00:00:00
Exposure Time [%]                    7.300509
Equity Final [$]               1459250.938163
Equity Peak [$]                1501558.738254
Return [%]                          45.925094
Buy & Hold Return [%]              133.833953
Return (Ann.) [%]                    5.537629
Volatility (Ann.) [%]                5.495878
Sharpe Ratio                         1.007597
Sortino Ratio                        2.252843
Calmar Ratio                         0.788559
Max. Drawdown [%]                   -7.022468
Avg. Drawdown [%]                   -1.711576
Max. Drawdown Duration      789 days 00:00:00
Avg. Drawdown Duration      118 days 00:00:00
# Trades                                   83
Win Rate [%]                        28.915663
Best Trade [%]                       5.224138
Worst Trade [%]                       -1.7254
Avg. Trade [%]                    

Row(id='12805', ...)

In [32]:
%%time

stats = bt.optimize(#ema_period= range(5, 200, 5),
                    target_val = range(5, 105, 5),
                    bound = range(1, 11, 1),
                    maximize='Equity Final [$]',
                    constraint = None)
                    #constraint=lambda param: param.target_val >= 4*param.bound)
stats

Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

CPU times: user 125 ms, sys: 297 ms, total: 422 ms
Wall time: 3.44 s


Start                     2015-01-01 00:00:00
End                       2022-02-18 00:00:00
Duration                   2605 days 00:00:00
Exposure Time [%]                    9.281268
Equity Final [$]               1726702.214348
Equity Peak [$]                1816356.637623
Return [%]                          72.670221
Buy & Hold Return [%]              133.833953
Return (Ann.) [%]                    8.101239
Volatility (Ann.) [%]                7.439639
Sharpe Ratio                         1.088929
Sortino Ratio                        2.248863
Calmar Ratio                         1.641273
Max. Drawdown [%]                   -4.935948
Avg. Drawdown [%]                   -1.600696
Max. Drawdown Duration      518 days 00:00:00
Avg. Drawdown Duration       71 days 00:00:00
# Trades                                   59
Win Rate [%]                        23.728814
Best Trade [%]                       6.091895
Worst Trade [%]                     -1.527664
Avg. Trade [%]                    

In [33]:
stats._strategy

<Strategy Rebound_Strategy(target_val=60,bound=2)>

RSI Divergences should also be useful

In [ ]:
df = pd.read_csv('./ICICIBANK.csv', index_col=0)
#df = pd.read_csv('./NIFTY50.csv')
for column in df.columns:
    if column != 'date':
       df[column] = pd.to_numeric(df[column])
#df['date'] = pd.to_datetime(df['date'])
#df['date'] = df['date'].tz_localize(None)
#df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df.index = df.index.tz_localize(None)

df = df.sort_index()
df = df.reindex(columns = ['open', 'high', 'low', 'close', 'volume'])
#df.drop('volume', axis=1, inplace=True)
df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close":"Close", "volume":'Volume'}, inplace=True)

df = df[~df.index.duplicated(keep='first')]
#Optionally, filter out by date range
#print(df.head(10))
filter = False
if filter:
    start_date = '2015-02-01 09:15:00'
    end_date = '2015-03-01 03:30:00'
    df = df.loc[start_date:end_date]

df.head()

In [ ]:
def divergence(Data, lower_barrier, upper_barrier, width):
    Data = adder(Data, 10)
    for i in range(len(Data)):
        try:
            if Data.iloc[i].rsi < lower_barrier:
                for a in range(i + 1, i + width):
                    # First trough
                    if Data.iloc[a].rsi > lower_barrier:
                        for r in range(a + 1, a + width):
                            if Data.iloc[r].rsi < lower_barrier and \
                            Data.iloc[r].rsi > Data.iloc[i].rsi and Data.iloc[r].Close < Data.iloc[i].Close:
                                for s in range(r + 1, r + width):
                                    # Second trough
                                    if Data.iloc[s].rsi > lower_barrier:
                                        Data.iloc[r].divergence = 1
                                        break
                                    else:
                                        break
                            else:
                                break
                        else:
                            break
                    else:
                        break
        except IndexError:
            pass
      
    for i in range(len(Data)):
        try:
            if Data.iloc[i].rsi > upper_barrier:
                for a in range(i + 1, i + width):
                    # First trough
                    if Data.iloc[a].rsi < upper_barrier:
                        for r in range(a + 1, a + width):
                            if Data.iloc[r].rsi > upper_barrier and \
                            Data.iloc[r].rsi < Data.iloc[i].rsi and Data.iloc[r].Close > Data.iloc[i].Close:
                                for s in range(r + 1, r + width):
                                    # Second trough
                                    if Data.iloc[s].rsi < upper_barrier:
                                        Data.iloc[s].divergence = -1
                                        break
                                    else:
                                        break
                            else:
                                break
                        else:
                            break
                    else:
                        break
        except IndexError:
            pass 
    return Data

In [ ]:
rsi_period = 14
df['rsi'] = talib.RSI(df.Close, rsi_period)
df['divergence'] = 0
df = divergence(df, lower_barrier, upper_barrier, width)

Lets experiment with a self-made strategy.

In [34]:
class Trend_Follow_Custom(Strategy):
    '''
    Define for a time frame > 1 min:
    Advance Rate = # points in upmove/ # upmoves
    Decline rate = # points in downmove/ # downmoves
    
    Advance volatility = mean((High - Low)/(Close - Open))
    Decline volatility = mean((High - Low)/(Open - Close))
    
    Price Trend = Advance rate/ Decline Rate
    Volatility Trend = Advance volatility / Decline Volatility
    '''
    tf = 5 #minutes
    trend_up_thresh = 1.5
    trend_down_thresh = 1/1.5
    def init(self):
        self.t = 1
        pass
    
    def next(self):
        self.t += 1
        advances = 0
        declines = 0
        num_advances = 0
        num_declines = 0
        ar = 0
        dr = 0
        trend = 0
        min_val = self.data.Low[-1]
        max_val = self.data.High[-1]

        #print(self.t, self.data.Close[-1])
        if self.t%self.tf==0:
            #Compute the values for the last slot
            if self.t==self.tf: #Edge cases
                if (self.data.Close[-self.tf] > self.data.Open[-self.tf]):
                    advances += self.data.Close[-self.tf] - self.data.Open[-self.tf]
                    num_advances += 1
                    min_val = min(min_val, self.data.Low[-self.tf])
                if (self.data.Close[-self.tf] < self.data.Open[-self.tf]):
                    declines += - self.data.Close[-self.tf] + self.data.Open[-self.tf]
                    num_declines += 1
                    max_val = max(max_val, self.data.High[-self.tf+t])
            else:
                if (self.data.Close[-self.tf] > self.data.Close[-self.tf-1]):
                    advances += self.data.Close[-self.tf] - self.data.Close[-self.tf-1]
                    num_advances += 1
                    min_val = min(min_val, self.data.Low[-self.tf])
                if (self.data.Close[-self.tf] < self.data.Close[-self.tf-1]):
                    declines += - self.data.Close[-self.tf] + self.data.Close[-self.tf-1]
                    num_declines += 1
                    max_val = max(max_val, self.data.High[-self.tf])
            for t in range(0, self.tf-1):
                #print(self.data.Close[-self.tf+t])
                if (self.data.Close[-self.tf+t+1] > self.data.Close[-self.tf+t]):
                    advances += self.data.Close[-self.tf+t+1] - self.data.Close[-self.tf+t]
                    num_advances += 1
                    min_val = min(min_val, self.data.Low[-self.tf+t])
                if (self.data.Close[-self.tf+t+1] < self.data.Close[-self.tf+t]):
                    declines += - self.data.Close[-self.tf+t+1] + self.data.Close[-self.tf+t]
                    num_declines += 1
                    max_val = max(max_val, self.data.High[-self.tf+t])
            #print(self.data.Close[-1])
            if num_advances>0:
                ar = advances/num_advances
            if num_declines>0:
                dr = declines/num_declines
            if dr>0:
                trend = ar/dr
            else:
                trend = ar/0.1
            print(advances, num_advances, declines, num_declines, ar, dr, trend)
            if trend > self.trend_up_thresh and self.data.Close[-1]>min_val: #If we are going down, then don't enter trade
                if not self.position:
                    print(f'Long {self.data.Close[-1]}. SL: {min_val}')
                    self.buy(sl = min_val)
                else:
                    for trade in self.trades:
                        print(f'Update SL: {min_val}')
                        trade.sl = min_val #Update stop loss
            elif trend < self.trend_down_thresh and self.data.Close[-1]<max_val: #If we are going up, then don't enter trade:
                if not self.position:
                    print(f'Short {self.data.Close[-1]}. SL: {min_val}')
                    self.sell(sl = max_val)
                else:
                    for trade in self.trades:
                        print(f'Update SL: {max_val}')
                        trade.sl = max_val #Update stop loss
from backtesting import Backtest
bt = Backtest(df, Trend_Follow_Custom, cash=1000000, commission=0/1000000, trade_on_close=False)
stats = bt.run()
print(stats)

bt.plot()

57.5 3 30.049999999999272 2 19.166666666666668 15.024999999999636 1.2756516916251004
22.599999999998545 4 7.599999999998545 1 5.649999999999636 7.599999999998545 0.7434210526316735
24.299999999999272 3 18.75 2 8.099999999999758 9.375 0.8639999999999742
34.75 4 6.599999999998545 1 8.6875 6.599999999998545 1.316287878788169
12.850000000002183 1 33.900000000001455 4 12.850000000002183 8.475000000000364 1.516224188790753
Long 17291.15. SL: 17275.0
0.25 1 32.55000000000291 4 0.25 8.137500000000728 0.030721966205834426
Short 17258.85. SL: 17258.15
22.950000000000728 3 26.0 2 7.650000000000243 13.0 0.5884615384615571
Update SL: 17264.6
13.099999999998545 3 15.999999999996362 2 4.366666666666181 7.999999999998181 0.5458333333333968
Short 17252.9. SL: 17244.05
25.599999999998545 4 3.900000000001455 1 6.399999999999636 3.900000000001455 1.6410256410249355
Long 17274.6. SL: 17252.9
29.0 4 1.8499999999985448 1 7.25 1.8499999999985448 3.9189189189220013
Update SL: 17273.75
5.349999999998545 1 17.25

Row(id='9688', ...)